1. Для нашего пайплайна (Case1) поэкспериментировать с разными моделями: 1 - бустинг, 2 - логистическая регрессия (не забудьте здесь добавить в cont_transformer стандартизацию - нормирование вещественных признаков)
2. Отобрать лучшую модель по метрикам (кстати, какая по вашему мнению здесь наиболее подходящая DS-метрика)
3. Для отобранной модели (на отложенной выборке) сделать оценку экономической эффективности при тех же вводных, как в вопросе 2 (1 доллар на привлечение, 2 доллара - с каждого правильно классифицированного (True Positive) удержанного). (подсказка) нужно посчитать FP/TP/FN/TN для выбранного оптимального порога вероятности и посчитать выручку и траты.
4. (опционально) Провести подбор гиперпараметров лучшей модели по итогам 2-3
5. (опционально) Еще раз провести оценку экономической эффективности

In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('churn_data.csv')
df.head(3)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


In [3]:
X_train, X_test, y_train, y_test = train_test_split(df, df['Exited'], random_state=0)

In [4]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]

In [5]:
categorical_columns = ['Geography', 'Gender', 'Tenure', 'HasCrCard', 'IsActiveMember']
continuous_columns = ['CreditScore', 'Age', 'Balance', 'NumOfProducts', 'EstimatedSalary']

In [6]:
final_transformers = list()

for cat_col in categorical_columns:
    cat_transformer = Pipeline([
                ('selector', FeatureSelector(column=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    final_transformers.append((cat_col, cat_transformer))
    
for cont_col in continuous_columns:
    cont_transformer = Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    final_transformers.append((cont_col, cont_transformer))

In [7]:
feats = FeatureUnion(final_transformers)

In [8]:
def fit_model(model, params=None):
    
    pipeline = Pipeline([
        ('features',feats),
        ('classifier', model)
        ])
    
    if params:
        grid = GridSearchCV(pipeline,
                    param_grid=params,
                    cv=4,
                    refit=False)
        search = grid.fit(X_train, y_train)
        print(search.best_params_)
        pipeline.set_params(**search.best_params_)
        pipeline.fit(X_train, y_train)
    
    else: 
        pipeline.fit(X_train, y_train)
    
    preds = pipeline.predict_proba(X_test)[:, 1]
    
    precision, recall, thresholds = precision_recall_curve(y_test, preds)
    fscore = (2 * precision * recall) / (precision + recall)
    ix = np.argmax(fscore)
    cnf_matrix = confusion_matrix(y_test, preds>thresholds[ix])
    
    profit = cnf_matrix[1][1] * 2 - (cnf_matrix[0][1] + cnf_matrix[1][1]) * 1
    print('profit=%d, Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f, %s' % (profit,
                                                                                           thresholds[ix],
                                                                                           fscore[ix],
                                                                                           precision[ix],
                                                                                           recall[ix],
                                                                                           type(model).__name__))
    return model

In [9]:
models = [CatBoostClassifier(random_state = 1, verbose=False),
         RandomForestClassifier(random_state = 1),
         LogisticRegression(random_state = 1)]

for model in models:
    fit_model(model)

profit=116, Best Threshold=0.307055, F-Score=0.645, Precision=0.598, Recall=0.699, CatBoostClassifier
profit=143, Best Threshold=0.350000, F-Score=0.651, Precision=0.628, Recall=0.676, RandomForestClassifier
profit=-48, Best Threshold=0.289522, F-Score=0.510, Precision=0.462, Recall=0.568, LogisticRegression


При низком precision большая ошибка FP, что приведет к ненужным расходам на лояльных клиентов. При низком recall большая ошибка FN, значит кленты, готовые уйти, не получат внимания или предложения. Тут нужно искать компромисс, исходя из политики бизнеса (в том числе бюджетной). Модель логистической регресси показывает наименьшие ds метрики и убыточна. Посмотрим на две другие модели.

In [10]:
params={'classifier__depth':[2, 3, 4],
        'classifier__min_data_in_leaf':[1, 2, 4]
        }
fit_model(CatBoostClassifier(iterations=300, learning_rate=0.09, random_state = 1, verbose=False), params)

{'classifier__depth': 3, 'classifier__min_data_in_leaf': 1}
profit=162, Best Threshold=0.375798, F-Score=0.651, Precision=0.669, Recall=0.633, CatBoostClassifier


In [11]:
params={'classifier__max_features':[0.2, 0.3, 0.7],
        'classifier__min_samples_leaf':[2, 4, 5]
        }
fit_model(RandomForestClassifier(n_estimators=100, random_state = 1), params)

{'classifier__max_features': 0.3, 'classifier__min_samples_leaf': 4}
profit=158, Best Threshold=0.349499, F-Score=0.659, Precision=0.654, Recall=0.664, RandomForestClassifier


RandomForestClassifier(max_features=0.3, min_samples_leaf=4, random_state=1)

В обоих случаях прибыль стала выше, precision заметно растет, и падает recall. Немног выше стал F-score. В данной задаче максимизация precision позволяет не тратить деньги на ложно-положительные объекты, что приносит больше прибыли.